In [ ]:
!pip install 'bs4==0.0.1' 'lxml==4.9.2' 'googlemaps==4.10.0'

In [81]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.parse import unquote
import json
import time
import os
import googlemaps

# API key for places, directions, geocode and another to maps js
gmaps = googlemaps.Client(key=os.environ.get("GOOGLE_KEY"))

locations = []
geo = []
names = []
url = []
food = []
food_desc = []

def get_google(name, query, retry=3):
    try:
        result = gmaps.geocode(unquote(query))
        if len(result) > 1:
            place = gmaps.places(unquote(name))["results"]
            if len(place) == 1:
                result = place
        if not result: raise Exception()
        return result
    except Exception as e:
        if retry:
            time.sleep(1)
            return get_google(name, query, retry-1)
        else:
            print(f"query error {query}")
            print(e)
            return False

for i in range(1, 80):
    req = Request(f"http://www.comidadibuteco.com.br/category/butecos/page/{i}/")
    html_page = urlopen(req)

    soup = BeautifulSoup(html_page, "lxml")

    inner = soup.findAll('div', {"class": "result-inner"})
    for div in inner:
        name = div.find('h2').text
        links = div.findAll('a')
        if name and (len(links) == 2):
            u = links[0].get('href')
            r = Request(u)
            html_bar = urlopen(r)
            s = BeautifulSoup(html_bar, "lxml")
            data = s.find('section', {"class": "mecanica"})
            ps = data.find('div', {'class': 'text-left'}).findAll('p')
            addr = False
            for p in ps:
                if "Endereço" in p.text:
                    addr = p
            if not addr:
                print(f"no addr found for {name}")
                continue
            geocode_result = get_google(name, ", ".join(addr.text.split("Endereço:")[1:]).strip())
            if geocode_result:
                geo.append(geocode_result[0]["geometry"]["location"])
            else:
                print(f"no place found for {name}")
                geo.append({})
            url.append(u)
            names.append(name)
            flist = ps[0].get_text(separator='|br|', strip=True).split('|br|')
            food.append(flist[0])
            if len(flist) > 1:
                food_desc.append(flist[1])
            else:
                print(f"no description found for {name}")
                food_desc.append("")

        else:
            continue
    print(f"foi página {i}")
    time.sleep(10)
for i in range(len(names)):
    locations.append({
        "name": names[i],
        "geo": geo[i],
        "url": url[i],
        "food": food[i],
        "food_desc": food_desc[i]
    })

no description found for 1960 bar
no description found for 20 Busca Bar
no description found for A Virgem Bar
foi página 1
no description found for Alek Bar
no description found for Alquimia do Sabor
foi página 2
no description found for Armazém do Alemão
no description found for Armazém Porto Alegre
foi página 3
no description found for Arrojado’s Bar
foi página 4
no description found for Baldhead Cervejaria e Bar
foi página 5
no description found for Bar Chopp Petiscos
no description found for Bar Cruzeiro
foi página 6
no description found for Bar da Oma
foi página 7
no description found for Bar do Alemão
foi página 8
no description found for Bar do Cação
no description found for Bar do Chicão
no description found for Bar do Clebinho
foi página 9
foi página 10
foi página 11
no description found for Bar do Macário
foi página 12
no description found for Bar do Paulinho
no description found for Bar do Ponce
no description found for Bar do Rafa
foi página 13
foi página 14
no description 

In [82]:
with open("places.json", "w") as f:
    f.write(json.dumps(locations, ensure_ascii=False))

In [83]:
len(names)

937

In [84]:
len(geo)

937

In [85]:
len(url)

937

In [86]:
len(food)

937

In [87]:
len(food_desc)

937

In [88]:
len(list(filter(lambda item: bool(item), geo)))

937